In [1]:
import gym_examples
import gym
import numpy as np
# from stable_baselines3 import PPO
import sb3_contrib as sb3
from sb3_contrib import MaskablePPO
from sb3_contrib.common.wrappers import ActionMasker
from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy
from gym.wrappers import FlattenObservation
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import SubprocVecEnv
from sb3_contrib.common.maskable.utils import get_action_masks

In [2]:
sb3.__version__, gym.__version__

('1.7.0', '0.26.0')

# CNN ENV TESTING

In [3]:
env2 = gym.make("gym_examples/CNN-v0")

In [4]:
start_cnn = env2.reset()
start_cnn

({'layer_type': 0,
  'layer_depth': 0,
  'filter_depth': 0,
  'filter_size': 0,
  'fc_size': 0,
  'is_start': 1,
  'pool_size_and_stride': 0},
 {'current_network': [{'layer_type': 'conv',
    'layer_depth': 0,
    'filter_depth': 0,
    'filter_size': 0,
    'fc_size': 0,
    'image_size': 28,
    'pool_size': 0,
    'pool_stride': 0,
    'is_start': 1}],
  'current_image_size': 28,
  'current_layer_depth': 0,
  'current_num_fc_layers': 0,
  'obs_vector': array([0, 0, 0, 0, 0, 1, 0])})

In [5]:
env2.action_space

MultiDiscrete([3 9 5 4 5 2 4])

In [6]:
env2.get_valid_action_mask()

inside layer depth limit
enabling convolutions


(array([1, 0, 0], dtype=int8),
 array([0, 1, 0, 0, 0, 0, 0, 0, 0], dtype=int8),
 array([0, 1, 1, 1, 1], dtype=int8),
 array([0, 1, 1, 1], dtype=int8),
 array([1, 0, 0, 0, 0], dtype=int8),
 array([1, 0], dtype=int8),
 array([1, 0, 0, 0], dtype=int8))

In [7]:
env2.action_space.sample(mask=env2.get_valid_action_mask())

inside layer depth limit
enabling convolutions


array([0, 1, 4, 2, 0, 0, 0], dtype=int64)

In [7]:
# take an action that picks a conv layer, with 20 filters with a 3x3 kernel. 
action = np.array([0, 1, 2, 2, 0, 0, 0], dtype=np.int64)

new_obs, reward, terminated, _, info = env2.step(action)

inside layer depth limit
enabling convolutions
enabling valid pools


In [8]:
new_obs

{'layer_type': 0,
 'layer_depth': 1,
 'filter_depth': 2,
 'filter_size': 2,
 'fc_size': 0,
 'is_start': 0,
 'pool_size_and_stride': 0}

In [9]:
info

{'current_network': [{'layer_type': 'conv',
   'layer_depth': 0,
   'filter_depth': 0,
   'filter_size': 0,
   'fc_size': 0,
   'image_size': 28,
   'pool_size': 0,
   'pool_stride': 0,
   'is_start': 1},
  {'layer_type': 'conv',
   'layer_depth': 1,
   'filter_depth': 20,
   'filter_size': 3,
   'fc_size': 0,
   'image_size': 26,
   'is_start': 0,
   'pool_size': 0,
   'pool_stride': 0}],
 'current_image_size': 26,
 'current_layer_depth': 1,
 'current_num_fc_layers': 0,
 'obs_vector': array([0, 1, 2, 2, 0, 0, 0])}

In [11]:
env2.get_valid_action_mask()

inside layer depth limit
enabling convolutions
enabling valid pools


(array([1, 1, 1], dtype=int8),
 array([0, 0, 1, 0, 0, 0, 0, 0, 0], dtype=int8),
 array([0, 1, 1, 1, 1], dtype=int8),
 array([0, 1, 1, 1], dtype=int8),
 array([0, 1, 1, 1, 1], dtype=int8),
 array([1, 1], dtype=int8),
 array([0, 1, 1, 1], dtype=int8))

In [12]:
#make an fc layer with 512
action = np.array([2, 2, 2, 2, 1, 0, 1], dtype=np.int64)
new_obs, reward, terminated, _, info = env2.step(action)

info

{'current_network': [{'layer_type': 'conv',
   'layer_depth': 0,
   'filter_depth': 0,
   'filter_size': 0,
   'fc_size': 0,
   'image_size': 28,
   'pool_size': 0,
   'pool_stride': 0,
   'is_start': 1},
  {'layer_type': 'conv',
   'layer_depth': 1,
   'filter_depth': 20,
   'filter_size': 3,
   'fc_size': 0,
   'image_size': 26,
   'is_start': 0,
   'pool_size': 0,
   'pool_stride': 0},
  {'layer_type': 'fc',
   'layer_depth': 2,
   'filter_depth': 20,
   'filter_size': 3,
   'fc_size': 512,
   'image_size': 26,
   'is_start': 0,
   'pool_size': 5,
   'pool_stride': 3}],
 'current_image_size': 26,
 'current_layer_depth': 2,
 'current_num_fc_layers': 1}

In [13]:
env2.get_valid_action_mask() #Only fc layer allowed and fc layer with less than 512 nodes and this has to be terminal layer

inside layer depth limit


(array([0, 0, 1], dtype=int8),
 array([0, 0, 0, 1, 0, 0, 0, 0, 0], dtype=int8),
 array([1, 0, 0, 0, 0], dtype=int8),
 array([1, 0, 0, 0], dtype=int8),
 array([0, 0, 1, 1, 1], dtype=int8),
 array([0, 1], dtype=int8),
 array([1, 0, 0, 0], dtype=int8))

In [14]:
action = np.array([2, 3, 0, 0, 2, 1, 0], dtype=np.int64)
new_obs, reward, terminated, _, info = env2.step(action)

info

network =  [{'layer_type': 'conv', 'layer_depth': 0, 'filter_depth': 0, 'filter_size': 0, 'fc_size': 0, 'image_size': 28, 'pool_size': 0, 'pool_stride': 0, 'is_start': 1}, {'layer_type': 'conv', 'layer_depth': 1, 'filter_depth': 20, 'filter_size': 3, 'fc_size': 0, 'image_size': 26, 'is_start': 0, 'pool_size': 0, 'pool_stride': 0}, {'layer_type': 'fc', 'layer_depth': 2, 'filter_depth': 20, 'filter_size': 3, 'fc_size': 512, 'image_size': 26, 'is_start': 0, 'pool_size': 5, 'pool_stride': 3}, {'layer_type': 'fc', 'layer_depth': 3, 'filter_depth': 0, 'filter_size': 0, 'fc_size': 256, 'image_size': 26, 'is_start': 0, 'pool_size': 0, 'pool_stride': 0}] terminated = True
layersList =  [('conv', 1, 20, 3, 1, 26, 0, 0, []), ('fc', 2, 20, 5, 3, 26, 512, 0, []), ('fc', 3, 0, 0, 0, 26, 256, 0, [])]


{'current_network': [{'layer_type': 'conv',
   'layer_depth': 0,
   'filter_depth': 0,
   'filter_size': 0,
   'fc_size': 0,
   'image_size': 28,
   'pool_size': 0,
   'pool_stride': 0,
   'is_start': 1},
  {'layer_type': 'conv',
   'layer_depth': 1,
   'filter_depth': 20,
   'filter_size': 3,
   'fc_size': 0,
   'image_size': 26,
   'is_start': 0,
   'pool_size': 0,
   'pool_stride': 0},
  {'layer_type': 'fc',
   'layer_depth': 2,
   'filter_depth': 20,
   'filter_size': 3,
   'fc_size': 512,
   'image_size': 26,
   'is_start': 0,
   'pool_size': 5,
   'pool_stride': 3},
  {'layer_type': 'fc',
   'layer_depth': 3,
   'filter_depth': 0,
   'filter_size': 0,
   'fc_size': 256,
   'image_size': 26,
   'is_start': 0,
   'pool_size': 0,
   'pool_stride': 0}],
 'current_image_size': 26,
 'current_layer_depth': 3,
 'current_num_fc_layers': 2}

In [15]:
reward

0.9907848744836352

In [5]:
# env = gym.make("gym_examples/StarWars-v0")

# PPO DEBUGGING

In [2]:
myspace = gym.spaces.MultiDiscrete([4, 8, 5, 4, 4, 2])
myspace

MultiDiscrete([4 8 5 4 4 2])

In [3]:
# md_mask = tuple([np.ones(x,) for x in myspace.nvec])
# md_mask

In [4]:
# mysample = myspace.sample()
# mysample

In [5]:
# array([1, 6, 4, 2, 2, 0]) -> interpreted as [layer_type, layer_depth, filter_depth, filter_size, fc_size, terminal]
# i.e. [pool, 6th layer, 512 filter depth, 3 kernel size, 128 FC size, terminal]


In [6]:
# flatten(myspace, mysample)

In [7]:
cnn_env = gym.make("gym_examples/CNN-v0")
cnn_env.verbose = True

In [8]:
# cnn_env = ActionMasker(cnn_env, masking_fn)

In [9]:
# cnn_env = ActionMasker(FlattenObservation(cnn_env, masking_fn))

In [10]:
cnn_env.observation_space.sample()

OrderedDict([('fc_size', 4),
             ('filter_depth', 0),
             ('filter_size', 2),
             ('is_start', 1),
             ('layer_depth', 7),
             ('layer_type', 0),
             ('pool_size_and_stride', 0)])

In [11]:
# cnn_env = FlattenObservation(cnn_env)
# cnn_env = make_vec_env(cnn_env, n_envs=2)

In [12]:
model = MaskablePPO(MaskableActorCriticPolicy, FlattenObservation(cnn_env), verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [13]:
model.learn(total_timesteps=100)

network =  [{'layer_type': 'conv', 'layer_depth': 1, 'filter_depth': 64, 'filter_size': 3, 'fc_size': 0, 'image_size': 26, 'is_start': 0, 'pool_size': 0, 'pool_stride': 0}, {'layer_type': 'pool', 'layer_depth': 2, 'filter_depth': 20, 'filter_size': 1, 'fc_size': 128, 'image_size': 13, 'is_start': 0, 'pool_size': 2, 'pool_stride': 2}, {'layer_type': 'fc', 'layer_depth': 3, 'filter_depth': 32, 'filter_size': 5, 'fc_size': 256, 'image_size': 13, 'is_start': 0, 'pool_size': 0, 'pool_stride': 0}] terminated = True
network =  [{'layer_type': 'conv', 'layer_depth': 1, 'filter_depth': 64, 'filter_size': 3, 'fc_size': 0, 'image_size': 26, 'is_start': 0, 'pool_size': 0, 'pool_stride': 0}] terminated = True
network =  [{'layer_type': 'conv', 'layer_depth': 1, 'filter_depth': 10, 'filter_size': 3, 'fc_size': 0, 'image_size': 26, 'is_start': 0, 'pool_size': 0, 'pool_stride': 0}] terminated = True
network =  [{'layer_type': 'conv', 'layer_depth': 1, 'filter_depth': 32, 'filter_size': 5, 'fc_size': 0

In [ ]:
# # env = ActionMasker(env, masking_fn)
# model = MaskablePPO(MaskableActorCriticPolicy, cnn_env, verbose=1)
# model.learn(total_timesteps=(100))